In [1]:
%pip install ../../

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=81864 sha256=b9feebd435a93907f5998d748a7864c841c4d9433c3acf625ee402ab38ee36af
  Stored in directory: /tmp/pip-ephem-wheel-cache-3czse4uc/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [3]:
from spuco.utils import set_seed

set_seed(0)

In [4]:
from wilds import get_dataset
import torchvision.transforms as transforms

# Load the full dataset, and download it if necessary
dataset = get_dataset(dataset="waterbirds", download=True)

transform = transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])

# Get the training set
train_data = dataset.get_subset(
    "train",
    transform=transform
)

# Get the training set
test_data = dataset.get_subset(
    "test",
    transform=transform
)

In [5]:
from spuco.utils import WILDSDatasetWrapper

trainset = WILDSDatasetWrapper(dataset=train_data, metadata_spurious_label="background", verbose=True)
testset = WILDSDatasetWrapper(dataset=test_data, metadata_spurious_label="background", verbose=True)

Partitioning data indices into groups: 100%|██████████| 5794/5794 [00:00<00:00, 2518060.03it/s]


In [6]:
trainset.group_weights

{(1, 1): 0.22043795620437956,
 (1, 0): 0.01167883211678832,
 (0, 0): 0.7295099061522419,
 (0, 1): 0.0383733055265902}

In [7]:
from spuco.models import model_factory 

model = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)

In [8]:
from torch.optim import SGD
from spuco.robust_train import ERM 

erm = ERM(
    model=model,
    num_epochs=1,
    trainset=trainset,
    batch_size=128,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
erm.train()

Epoch 0: 100%|██████████| 38/38 [00:04<00:00,  8.58batch/s, accuracy=84.7457627118644%, loss=0.396]


In [9]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:  25%|██▌       | 1/4 [00:02<00:06,  2.09s/it]

Group (0, 0) Test Accuracy: 92.63858093126386


Evaluating group-wise accuracy:  50%|█████     | 2/4 [00:04<00:04,  2.06s/it]

Group (0, 1) Test Accuracy: 50.90909090909091


Evaluating group-wise accuracy:  75%|███████▌  | 3/4 [00:05<00:01,  1.55s/it]

Group (1, 0) Test Accuracy: 12.305295950155763


Evaluating group-wise accuracy: 100%|██████████| 4/4 [00:05<00:00,  1.50s/it]

Group (1, 1) Test Accuracy: 61.52647975077882


{(0, 0): 92.63858093126386,
 (0, 1): 50.90909090909091,
 (1, 0): 12.305295950155763,
 (1, 1): 61.52647975077882}